# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json
from pprint import pprint

# Import API key
from api_keys_vaccation import geoapify_key
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,24.09,88,20,4.63,TO,1682082267
1,1,parad,47.9232,20.0297,15.47,50,100,3.74,HU,1682082268
2,2,port alfred,-33.5906,26.8910,20.10,59,6,3.58,ZA,1682082269
3,3,hermanus,-34.4187,19.2345,16.56,52,100,3.14,ZA,1682082271
4,4,povenets,62.8487,34.8262,4.08,93,100,2.55,RU,1682082272
...,...,...,...,...,...,...,...,...,...,...
558,558,vila franca do campo,37.7167,-25.4333,16.23,64,78,7.59,PT,1682082994
559,559,puerto quijarro,-17.7833,-57.7667,24.80,53,1,1.77,BO,1682082996
560,560,bandar,2.0500,99.7500,24.77,97,100,1.28,ID,1682082975
561,561,along,28.1667,94.7667,22.49,69,48,0.74,IN,1682082999


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color= 'City',
    size = 'Humidity',
    scale = 1,
    alpha = 0.5
)

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filt = (city_data_df['Max Temp'] > 21) &  (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
city_fit_criteria = city_data_df[filt]

city_fit_criteria


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,sao sebastiao,-23.7600,-45.4097,22.37,56,0,0.09,BR,1682082313
147,147,lima,-12.0432,-77.0282,23.28,80,0,3.60,PE,1682082243
231,231,charleston,32.7766,-79.9309,23.10,88,0,3.09,US,1682082581
273,273,arcachon,44.5875,-1.0519,23.01,30,0,2.06,FR,1682082641
348,348,mount pleasant,32.7941,-79.8626,23.33,91,0,3.09,US,1682082511
360,360,puerto escondido,15.8500,-97.0667,25.46,83,0,0.00,MX,1682082568
402,402,pimentel,-6.8367,-79.9342,25.16,78,0,2.06,PE,1682082800
483,483,punta gorda,26.9298,-82.0454,24.43,83,0,4.02,US,1682082898


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_fit_criteria.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] = ""


# Display sample data
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
36,36,sao sebastiao,-23.7600,-45.4097,22.37,56,0,0.09,BR,1682082313,
147,147,lima,-12.0432,-77.0282,23.28,80,0,3.60,PE,1682082243,
231,231,charleston,32.7766,-79.9309,23.10,88,0,3.09,US,1682082581,
273,273,arcachon,44.5875,-1.0519,23.01,30,0,2.06,FR,1682082641,
348,348,mount pleasant,32.7941,-79.8626,23.33,91,0,3.09,US,1682082511,
360,360,puerto escondido,15.8500,-97.0667,25.46,83,0,0.00,MX,1682082568,
402,402,pimentel,-6.8367,-79.9342,25.16,78,0,2.06,PE,1682082800,
483,483,punta gorda,26.9298,-82.0454,24.43,83,0,4.02,US,1682082898,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key  
}
# Print a message to follow up the hotel search
print("Starting hotel search")
#https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-64.1811,-31.4135,10000&bias=proximity:-64.1811,-31.4135&limit=20&apiKey=ff63c45a930a4669a2a614984271d0b5

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{hotel_df.loc[index,"Lng"]},{hotel_df.loc[index,"Lat"]},{radius}'
    params["bias"] = f'proximity:{hotel_df.loc[index,"Lng"]},{hotel_df.loc[index,"Lat"]}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
   
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
#Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
sao sebastiao - nearest hotel: Hotel Arrastão
lima - nearest hotel: Hotel Peach Garden
charleston - nearest hotel: Mills House Charleston, Curio Collection by Hilton
arcachon - nearest hotel: Hôtel All Suites Le Teich
mount pleasant - nearest hotel: Old Village Post House
puerto escondido - nearest hotel: Hotel Barlovento
pimentel - nearest hotel: Hospedaje Calle del Sol
punta gorda - nearest hotel: SpringHill Suites by Marriott Punta Gorda Harborside


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
36,36,sao sebastiao,-23.7600,-45.4097,22.37,56,0,0.09,BR,1682082313,Hotel Arrastão
147,147,lima,-12.0432,-77.0282,23.28,80,0,3.60,PE,1682082243,Hotel Peach Garden
231,231,charleston,32.7766,-79.9309,23.10,88,0,3.09,US,1682082581,"Mills House Charleston, Curio Collection by Hi..."
273,273,arcachon,44.5875,-1.0519,23.01,30,0,2.06,FR,1682082641,Hôtel All Suites Le Teich
348,348,mount pleasant,32.7941,-79.8626,23.33,91,0,3.09,US,1682082511,Old Village Post House
360,360,puerto escondido,15.8500,-97.0667,25.46,83,0,0.00,MX,1682082568,Hotel Barlovento
402,402,pimentel,-6.8367,-79.9342,25.16,78,0,2.06,PE,1682082800,Hospedaje Calle del Sol
483,483,punta gorda,26.9298,-82.0454,24.43,83,0,4.02,US,1682082898,SpringHill Suites by Marriott Punta Gorda Harb...


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot

map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color= 'City',
    size = 'Humidity',
    scale = 1,
    alpha = 0.5,
    frame_width = 700,
    frame_height = 500,
    hover_cols=['Hotel Name','Country']
    #hover_cols=['place', 'time']
)

# Display the map 
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)